In [14]:
 ! pip install faiss-cpu
 ! pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [16]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np

# Load dataset from a CSV file
data = pd.read_csv('all_titles.csv')

# Create a DataFrame with specific columns from the dataset
df = pd.DataFrame(data, columns=['Title', 'Cleaned_Title'])

# Convert the 'Title' column to a list of strings
title = df['Title'].astype(str).tolist()

# Initialize the sentence transformer model for encoding text
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")

# Encode the titles into vectors
vectors = encoder.encode(title)

# Get the dimensionality of the vectors
vector_dimension = vectors.shape[1]

# Create a FAISS index for L2 distance (Euclidean distance)
index = faiss.IndexFlatL2(vector_dimension)

# Convert vectors to float32 for compatibility with FAISS, then normalize them
vectors = vectors.astype(np.float32)
faiss.normalize_L2(vectors)

# Add the normalized vectors to the FAISS index
index.add(vectors)

# Define the search query
search_text = 'Sales Manager at XYZ'

# Encode the search query into a vector
search_vector = encoder.encode([search_text])

# Ensure the search vector is of type float32 and normalize it
search_vector = search_vector.astype(np.float32)
faiss.normalize_L2(search_vector.reshape(1, -1))

# Number of nearest neighbors to find
k = 4

# Perform the search
distances, ann = index.search(search_vector, k=k)

# Create a DataFrame with the distances and indices of the nearest neighbors
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

# Merge the results with the original DataFrame to get the titles of the nearest neighbors
merge = pd.merge(results, df, left_on='ann', right_index=True)

# Display the top results
merge.head()